In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [4]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [5]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'serviceVersion': '3.28.2025.1',
 'dwellTimeMS': '0',
 'avgWebHostCPUTotalLoad': 'N/A',
 'dataHitType': 'FROM_MEMORY',
 'Source': 'B11B',
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisp

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


In [6]:
for i in range(5):
    print("Our current number is {}, the next is {}".format(i, i+1))

Our current number is 0, the next is 1
Our current number is 1, the next is 2
Our current number is 2, the next is 3
Our current number is 3, the next is 4
Our current number is 4, the next is 5


## 1.2 What happens if there's an error in our query?

In [8]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2028')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2028'

In [9]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [400]>

In [10]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [12]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 293 (char 292)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [14]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"statusCode":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2028R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2547\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2168\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 807\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 261"}'

'{"status":200,"serviceVersion":"3.28.2025.1","dwellTimeMS":"0","avgWebHostCPUTotalLoad":"N/A","dataHitType":"FROM_MEMORY","Source":"B11B","result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [16]:
def get_grades(grades):
    result = pd.DataFrame()
    
    for grade in grades: 
        example_naep_query = (
            'https://www.nationsreportcard.gov/'
            'Dataservice/GetAdhocData.aspx?'
            f'type=data&subject=writing&grade={grade}&'
            'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
        naep_resp = requests.get(example_naep_query)
        
        try: 
            naep_resp_j = naep_resp.json()
            naep_resp_j
            naep_resp_d = pd.DataFrame(naep_resp_j['result'])
            result = pd.concat([result,naep_resp_d]) 
        
        except Exception as e:
            print(e)
    return result

In [17]:
# your code here

grades = ['4','8','12']

R = []

for grade in grades: 
    example_naep_query = (
        'https://www.nationsreportcard.gov/'
        'Dataservice/GetAdhocData.aspx?'
        f'type=data&subject=writing&grade={grade}&'
        'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
    naep_resp = requests.get(example_naep_query)
    
    try: 
        naep_resp_j = naep_resp.json()
        naep_resp_j
        naep_resp_d = pd.DataFrame(naep_resp_j['result'])
        # result = pd.concat([result,naep_resp_d]) 
        R.append(naep_resp_d)
    
    except Exception as e:
        print(e)
        
pd.concat(R)

Invalid control character at: line 1 column 293 (char 292)


{'status': 200,
 'serviceVersion': '3.28.2025.1',
 'dwellTimeMS': '15.6467',
 'avgWebHostCPUTotalLoad': 'N/A',
 'dataHitType': 'FROM_MEMORY',
 'Source': 'B11B',
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isSt

{'status': 200,
 'serviceVersion': '3.28.2025.1',
 'dwellTimeMS': '0',
 'avgWebHostCPUTotalLoad': 'N/A',
 'dataHitType': 'FROM_MEMORY',
 'Source': 'B11B',
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 3,
   'CohortLabel': 'Grade 12',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 12,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 141.256977963264,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 3,
   'CohortLabel': 'Grade 12',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 12,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 155.385916780351,
   'isStat

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0
0,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,1,Male,141.256978,1,0
1,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,2,Female,155.385917,1,0


# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [20]:
## get the key
API_KEY = "1Gh9eKod5_-vX1otu_KEzcFphmJC58Kwb2qsJbcKPn76WUI7eW6biYfFe6R6Pmx_7NRaaQraF-Co_eLwcjvLl-DGQ4A0wHE8Z6WlFm9Bum6lEFvh9wiLufavouMtZXYx"

In [21]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [22]:
yelp_genjson['businesses'][0]

{'id': 'wyV_NfYn4ZOfp_sHMDPcAw',
 'alias': 'bistro-at-six-hanover',
 'name': 'Bistro at Six',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/i4jvssUsxa79VYVLOD3TmQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bistro-at-six-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 2,
 'categories': [{'alias': 'lounges', 'title': 'Lounges'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'tradamerican', 'title': 'American'}],
 'rating': 4.0,
 'coordinates': {'latitude': 43.7001146, 'longitude': -72.2877078},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '6 E South St',
  'address2': None,
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['6 E South St', 'Hanover, NH 03755']},
 'phone': '+16036430600',
 'display_phone': '(603) 643-0600',
 'distance': 198.6517

In [23]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'wyV_NfYn4ZOfp_sHMDPcAw',
 'alias': 'bistro-at-six-hanover',
 'name': 'Bistro at Six',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/i4jvssUsxa79VYVLOD3TmQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bistro-at-six-hanover?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 2,
 'categories': [{'alias': 'lounges', 'title': 'Lounges'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'tradamerican', 'title': 'American'}],
 'rating': 4.0,
 'coordinates': {'latitude': 43.7001146, 'longitude': -72.2877078},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '6 E South St',
  'address2': None,
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['6 E South St', 'Hanover, NH 03755']},
 'phone': '+16036430600',
 'display_phone': '(603) 643-0600',
 'distance': 198.6517

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,False,https://www.yelp.com/biz/bistro-at-six-hanover...,2,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.0,"{'latitude': 43.7001146, 'longitude': -72.2877...",[],$$,"{'address1': '6 E South St', 'address2': None,...",+16036430600,(603) 643-0600,198.651788
1,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,256,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758
2,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,561,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160
3,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,430,"[{'alias': 'tradamerican', 'title': 'American'...",4.2,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,245.079232
4,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,30,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.2,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552


In [24]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
wyV_NfYn4ZOfp_sHMDPcAw,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,https://www.yelp.com/biz/bistro-at-six-hanover...,$$,+16036430600,(603) 643-0600
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
neBEWvgHNhsXIBImCYx_6A,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,https://www.yelp.com/biz/murphy-s-on-the-green...,$$,+16036434075,(603) 643-4075
34j_2nRCVQBTKafJvncZlg,34j_2nRCVQBTKafJvncZlg,pine-restaurant-hanover-2,PINE Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/c2SgUR...,https://www.yelp.com/biz/pine-restaurant-hanov...,$$$,+16036468000,(603) 646-8000


In [25]:
# change location
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"

my_location_tpe = "New York City,United States,10024"
yelp_genquery_tpe = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location_tpe)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_tpe = requests.get(yelp_genquery_tpe, headers = header)
yelp_genresp_tpe

yelp_genresp_tpe.json()

<Response [200]>

{'businesses': [{'id': 'h9nuvIu8TyrQcYy8J1AOxg',
   'alias': 'the-ellington-new-york-4',
   'name': 'The Ellington',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/rVD8_DWEjEJBnYVa3r_ggQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/the-ellington-new-york-4?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
   'review_count': 312,
   'categories': [{'alias': 'tradamerican', 'title': 'American'},
    {'alias': 'bars', 'title': 'Bars'},
    {'alias': 'venues', 'title': 'Venues & Event Spaces'}],
   'rating': 4.6,
   'coordinates': {'latitude': 40.800868, 'longitude': -73.968243},
   'transactions': ['pickup', 'delivery'],
   'price': '$$',
   'location': {'address1': '2745 Broadway',
    'address2': None,
    'address3': '',
    'city': 'New York',
    'zip_code': '10025',
    'country': 'US',
    'state': 'NY',
    'display_address': ['2745 Broadway', 'New York, NY 10025']},

In [26]:
yelp_genjson_tpe = yelp_genresp_tpe.json()

## turn JSON into usable data (DF)
yelp_gendf = pd.DataFrame(yelp_genjson_tpe['businesses'])
#list(yelp_gendf_marlow) # list columns
yelp_gendf['location1'] = yelp_gendf.location.apply(lambda loclist: loclist['address1'])
yelp_gendf[['alias', 'name', 'url', 'review_count', 'rating', 'location1', 'price']]

,alias,name,url,review_count,rating,location1,price
0,the-ellington-new-york-4,The Ellington,https://www.yelp.com/biz/the-ellington-new-yor...,312,4.6,2745 Broadway,$$
1,spiga-new-york-2,Spiga,https://www.yelp.com/biz/spiga-new-york-2?adju...,91,4.8,57 W 84th St,$$
2,jacobs-pickles-new-york,Jacob's Pickles,https://www.yelp.com/biz/jacobs-pickles-new-yo...,5831,4.2,509 Amsterdam Ave,$$$
3,madison-edgewater-edgewater,Madison - Edgewater,https://www.yelp.com/biz/madison-edgewater-edg...,147,4.8,4 Pembroke Pl,$$
4,maison-pickle-new-york,Maison Pickle,https://www.yelp.com/biz/maison-pickle-new-yor...,2096,4.2,2315 Broadway,$$$
5,nobody-told-me-new-york,Nobody Told Me,https://www.yelp.com/biz/nobody-told-me-new-yo...,288,4.3,951 Amsterdam Ave,$$
6,dagon-new-york,Dagon,https://www.yelp.com/biz/dagon-new-york?adjust...,299,4.2,2454 Broadway,$$$
7,moon-kee-new-york,Moon Kee,https://www.yelp.com/biz/moon-kee-new-york?adj...,89,4.2,2642 Broadway,$$
8,parla-new-york,Parla,https://www.yelp.com/biz/parla-new-york?adjust...,161,4.4,320 Columbus Ave,$$
9,the-grand-feast-new-york,The Grand Feast,https://www.yelp.com/biz/the-grand-feast-new-y...,55,4.7,854 Amsterdam Ave,$$


In [27]:
# change location
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"

my_location_tpe = "New York City, New York, 10024"
yelp_genquery_tpe = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location_tpe)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_tpe = requests.get(yelp_genquery_tpe, headers = header)
# yelp_genresp_tpe

# yelp_genresp_tpe.json()

yelp_genjson_tpe = yelp_genresp_tpe.json()

## turn JSON into usable data (DF)
yelp_gendf = pd.DataFrame(yelp_genjson_tpe['businesses'])
#list(yelp_gendf_marlow) # list columns
yelp_gendf['location1'] = yelp_gendf.location.apply(lambda loclist: loclist['address1'])
yelp_gendf[['alias', 'name', 'url', 'review_count', 'rating', 'location1', 'price']]

,alias,name,url,review_count,rating,location1,price
0,the-ellington-new-york-4,The Ellington,https://www.yelp.com/biz/the-ellington-new-yor...,312,4.6,2745 Broadway,$$
1,spiga-new-york-2,Spiga,https://www.yelp.com/biz/spiga-new-york-2?adju...,91,4.8,57 W 84th St,$$
2,jacobs-pickles-new-york,Jacob's Pickles,https://www.yelp.com/biz/jacobs-pickles-new-yo...,5831,4.2,509 Amsterdam Ave,$$$
3,madison-edgewater-edgewater,Madison - Edgewater,https://www.yelp.com/biz/madison-edgewater-edg...,147,4.8,4 Pembroke Pl,$$
4,maison-pickle-new-york,Maison Pickle,https://www.yelp.com/biz/maison-pickle-new-yor...,2096,4.2,2315 Broadway,$$$
5,nobody-told-me-new-york,Nobody Told Me,https://www.yelp.com/biz/nobody-told-me-new-yo...,288,4.3,951 Amsterdam Ave,$$
6,dagon-new-york,Dagon,https://www.yelp.com/biz/dagon-new-york?adjust...,299,4.2,2454 Broadway,$$$
7,moon-kee-new-york,Moon Kee,https://www.yelp.com/biz/moon-kee-new-york?adj...,89,4.2,2642 Broadway,$$
8,parla-new-york,Parla,https://www.yelp.com/biz/parla-new-york?adjust...,161,4.4,320 Columbus Ave,$$
9,the-grand-feast-new-york,The Grand Feast,https://www.yelp.com/biz/the-grand-feast-new-y...,55,4.7,854 Amsterdam Ave,$$


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [62]:
# your code here
biz_id = "xt4sa64WOrpJvZBDPNPNYg"


In [64]:
# look at reviews of business with this id
base_url_reviews = f'https://api.yelp.com/v3/businesses/{biz_id}/reviews'
yelp_genquery_reviews = (base_url_reviews)

## use requests to call the API
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp_reviews = requests.get(yelp_genquery_reviews, headers = header)
yelp_genresp_reviews

## then, look at structure of response
yelp_genjson_reviews = yelp_genresp_reviews.json()
yelp_genjson_reviews

<Response [200]>

{'reviews': [{'id': 'K8O5pzwE8-8yKLdAyZiJ5g',
   'url': 'https://www.yelp.com/biz/jacobs-pickles-new-york?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=K8O5pzwE8-8yKLdAyZiJ5g&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w',
   'text': "I love pickles, make my own pickles, order pickles when possible every where I go. Made a special trip to Jacob's pickles to get a pickle flight and try the...",
   'rating': 3,
   'time_created': '2025-03-24 13:26:04',
   'user': {'id': 'poPW9m3A8fAN91J204fPZA',
    'profile_url': 'https://www.yelp.com/user_details?userid=poPW9m3A8fAN91J204fPZA',
    'image_url': None,
    'name': 'Megan B.'}},
  {'id': 'qwJYJ6dIOtAgu9bAyyA7zg',
   'url': 'https://www.yelp.com/biz/jacobs-pickles-new-york?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=qwJYJ6dIOtAgu9bAyyA7zg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w',
   'text': 'I am coming back here!! The food is soo good and I